In [6]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
import statsmodels.api as sm
import tensorflow as tf
import os

In [7]:
# Altere o diretório atual para o diretório onde o arquivo está localizado

print(tf.test.is_gpu_available())

# caminho_completo = "/Users/odair/Downloads/Arquivos Airbnb/total_data.csv"

# Altere o diretório atual para o diretório onde o arquivo está localizado
# os.chdir(os.path.dirname(caminho_completo))

# df_Airbnb = pd.read_csv(caminho_completo)

#os.chdir("D:\\Airbnb")

#df_Airbnb = pd.read_csv("total_data.csv")

False


In [ ]:
# Verificar as colunas com valores NaN e a quantidade de registros NaN
quantidade_nan_por_coluna = df_Airbnb.isna().sum()

print("Colunas com valores NaN e suas quantidades:")
print(quantidade_nan_por_coluna)

In [ ]:
# Função para determinar as estações do ano
def estacao_do_ano(mes):
    if 3 <= mes <= 5:
        return 1 #'Primavera'
    elif 6 <= mes <= 8:
        return 2 #'Verão'
    elif 9 <= mes <= 11:
        return 3 #'Outono'
    else:
        return 4 #'Inverno'

In [ ]:
# Converter a coluna 'data_string' para o tipo de dado de data
df_Airbnb['last_scraped'] = pd.to_datetime(df_Airbnb['last_scraped'], format='%Y-%m-%d', errors='coerce')
df_Airbnb['Data'] = df_Airbnb['last_scraped']
df_Airbnb.set_index('Data', inplace=True)

label_encoder = LabelEncoder()

# Normalizando os dados
# Criar um codigo numerico para os campos texto
df_Airbnb['neighbourhood_cleansed_encoded'] = label_encoder.fit_transform(df_Airbnb['neighbourhood_cleansed'])
df_Airbnb['property_type_encoded'] = label_encoder.fit_transform(df_Airbnb['property_type'])
df_Airbnb['room_type_encoded'] = label_encoder.fit_transform(df_Airbnb['room_type'])

# Converter as colunas para um campo numerico
df_Airbnb['id'] = pd.to_numeric(df_Airbnb['id'], errors='coerce', downcast='integer')
df_Airbnb['host_id'] = pd.to_numeric(df_Airbnb['host_id'], errors='coerce', downcast='integer')
df_Airbnb['accommodates'] = pd.to_numeric(df_Airbnb['accommodates'], errors='coerce', downcast='integer')

# Remova o símbolo de dólar ($) e outros caracteres não numéricos da coluna 'price'
df_Airbnb['price'] = df_Airbnb['price'].str.replace('[\$,]', '', regex=True)
df_Airbnb['price'] = df_Airbnb['price'].astype(float)

# Criando novas variáveis
df_Airbnb['data_int'] = df_Airbnb['last_scraped'].dt.strftime('%Y%m%d').fillna(0).astype(int)
df_Airbnb['media_price'] = df_Airbnb.groupby('neighbourhood_cleansed_encoded')['price'].transform('mean')
df_Airbnb['mediana_price'] = df_Airbnb.groupby('neighbourhood_cleansed_encoded')['price'].transform('median')
df_Airbnb['desvio_padrao_price'] = df_Airbnb.groupby('neighbourhood_cleansed_encoded')['price'].transform('std')
df_Airbnb['maximo_price'] = df_Airbnb.groupby('neighbourhood_cleansed_encoded')['price'].transform('max')
df_Airbnb['minimo_price'] = df_Airbnb.groupby('neighbourhood_cleansed_encoded')['price'].transform('min')

df_Airbnb['growth_data_int'] = df_Airbnb['data_int'].pct_change()
df_Airbnb['growth_media_price'] = df_Airbnb['media_price'].pct_change()
df_Airbnb['price_quartiles'] = pd.qcut(df_Airbnb['price'], q=4, labels=False)

df_Airbnb['mes'] = df_Airbnb['last_scraped'].dt.month
df_Airbnb['estacao'] = df_Airbnb['mes'].apply(estacao_do_ano)




#df_Airbnb = pd.get_dummies(df_Airbnb, columns=['amenities'], prefix=['amenity'])

# Gerar DF contendo codigo e descricao 
df_neighbourhood = df_Airbnb[["neighbourhood_cleansed_encoded", "neighbourhood_cleansed"]]

df_property = df_Airbnb[["property_type_encoded", "property_type"]]

df_room_type = df_Airbnb[["room_type_encoded", "room_type"]]

df_neighbourhood

In [ ]:
df_Airbnb

In [ ]:
# Faz uma cópia do df_Airbnb
df = df_Airbnb.copy()


# Cria um novo df com as colunas desejadas
#df = df[["last_scraped","neighbourhood_cleansed_encoded", "property_type_encoded","room_type_encoded","accommodates",
#         "bathrooms","bedrooms","beds","mes","estacao","data_int","media_price","mediana_price","desvio_padrao_price",
#         "maximo_price","minimo_price","growth_media_price","price_quartiles", "price"]]
df = df[["last_scraped","neighbourhood_cleansed_encoded", "property_type_encoded","room_type_encoded","accommodates",
         "bathrooms","bedrooms","beds","data_int","media_price","mediana_price","desvio_padrao_price",
         "maximo_price","minimo_price","growth_media_price","price_quartiles","mes","estacao", "price"]]
df

In [ ]:
quantidade_nan_por_coluna = df.isna().sum()

# Imprima as colunas com valores NaN e a quantidade de registros NaN
print("Colunas com valores NaN e suas quantidades:")
print(quantidade_nan_por_coluna)

In [ ]:
# apagar os registros com NAN
df = df.dropna()

# Separando os dados entre treinamento e teste
Linhas = len(df)
start_train = 0
end_train = int(0.7 * Linhas)
start_val = end_train + 1
end_val = Linhas - 1

print("Linhas Treinamento.: ", end_train)
print("Linhas Teste.......: ", Linhas - end_train)
print("Linhas Total.......: ", Linhas)

df_train = df.iloc[start_train:end_train+1]
df_val = df.iloc[start_val:end_val+1]


#X = df_train.iloc[:, df_train.columns.get_loc("id"):df_train.columns.get_loc("estacao")+1]
#y = df_train["price"]

#X_val = df_val.iloc[:, df_val.columns.get_loc("id"):df_val.columns.get_loc("estacao")+1]
#y_val = df_val["price"]

X_train = df_train.iloc[:, df_train.columns.get_loc("neighbourhood_cleansed_encoded"):df_train.columns.get_loc("estacao")+1]
y_train = df_train["price"]

X_test = df_val.iloc[:, df_val.columns.get_loc("neighbourhood_cleansed_encoded"):df_val.columns.get_loc("estacao")+1]
y_test = df_val["price"]


In [ ]:
X_train.columns

In [ ]:

# Inicializando os 2 modelos
model1 =  DecisionTreeClassifier(criterion = "entropy"
                                     , max_depth = 80
                                     , min_samples_leaf = 400
                                     , min_samples_split = 9
                                     , max_leaf_nodes = 850)
model2 = LinearRegression()

model3 =  DecisionTreeRegressor(random_state=42)


model5 =  RandomForestRegressor(n_estimators=100, random_state=42)  

# Treinamento
progress_bar = tqdm(total=15, position=0, leave=True, desc="Treinamento do Modelo")

model1.fit(X_train, y_train)
progress_bar.update(1)

model2.fit(X_train, y_train)
progress_bar.update(1)

model3.fit(X_train, y_train)
progress_bar.update(1)

#model4.fit(X_train, y_train)
progress_bar.update(1)

model5.fit(X_train, y_train)
progress_bar.update(1)

# Predição na base de treinamento
fit_train1 = model1.predict(X_train)
progress_bar.update(1)
fit_train2 = model2.predict(X_train)
progress_bar.update(1)
fit_train3 = model3.predict(X_train)
progress_bar.update(1)
#fit_train4 = model4.predict(X_train)


progress_bar.update(1)
fit_train5 = model5.predict(X_train)
progress_bar.update(1)

# Predicao na base de teste
fit_test1 = model1.predict(X_test)
progress_bar.update(1)
fit_test2 = model2.predict(X_test)
progress_bar.update(1)
fit_test3 = model3.predict(X_test)
progress_bar.update(1)
#fit_test4 = model4.predict(X_test)

progress_bar.update(1)
fit_test5 = model5.predict(X_test)
progress_bar.update(1)

# Feche a barra de progresso quando estiver completa
progress_bar.close()



In [ ]:
# Calculando a acurária do modelo
# Como o alvo é um campo float (price), vamos calcular a acurácia manualmente.
# As IA's tem funções para calculo de mse, msa, etc... para alvos binários
def Acuracia(_df1,_df2,_modelo):
    df_acuracia = pd.DataFrame(_df1)
    df_acuracia["Data"] = _df2.index
    df_acuracia.rename(columns={0: 'Alvo'}, inplace=True)
    df_acuracia.set_index('Data', inplace=True)
    
    df_result = pd.concat([_df2, df_acuracia], axis=1)
    df_result["Acertou"] = np.where((df_result['Alvo'] >= (df_result['price'] * 0.98) ) & 
                                    (df_result['Alvo'] <= (df_result['price'] * 1.02)  ), 1, 0)   
    print(f"{_modelo}..: {df_result['Acertou'].sum() / df_result['Acertou'].count() * 100:.2f}%, {df_result['Acertou'].sum()} / {df_result['Acertou'].count()}")



In [ ]:
df_train = df.iloc[start_train:end_train+1]
df_test = df.iloc[start_val:end_val+1]


Acuracia(fit_train1,df_train,"Base de Treinamento, modelo 1")
Acuracia(fit_train2,df_train,"Base de Treinamento, modelo 2")
Acuracia(fit_train3,df_train,"Base de Treinamento, modelo 3")
#Acuracia(fit_train4,df_train,"Base de Treinamento, modelo 4")
Acuracia(fit_train5,df_train,"Base de Treinamento, modelo 5")
print("=====================")
Acuracia(fit_test1,df_test,"Base de Teste, modelo 1")
Acuracia(fit_test2,df_test,"Base de Teste, modelo 2")
Acuracia(fit_test3,df_test,"Base de Teste, modelo 3")
#Acuracia(fit_test4,df_test,"Base de Teste, modelo 4")
Acuracia(fit_test5,df_test,"Base de Teste, modelo 4")


In [ ]:
=============

In [ ]:
# Salvar o modelo
import pickle


# Modelo 1 apresenta a melhor acuracia
caminho_completo = "/Users/odair/Downloads/Arquivos Airbnb/Modelo_Airbnb.pkl"

pickle.dump(model1,open(caminho_completo, 'wb'))

# Salvar modelo em windows
#pickle.dump(model4,open('c:\\Users\\odair\\Python-MT5\\Modelo_Airbnb.pkl', 'wb'))

In [ ]:
# importar o modelo para predicao

meu_arquivo = open('/Users/odair/Downloads/Arquivos Airbnb/Modelo_Airbnb.pkl',  'rb')
modelo = pickle.load(meu_arquivo)

In [ ]:
def busca_neighbourhood(nome):
    id = df_neighbourhood[df_neighbourhood['neighbourhood_cleansed'] == nome]['neighbourhood_cleansed_encoded'].values[0]
    return id


In [ ]:
def busca_property_type(nome):
    id = df_property[df_property['property_type'] == nome]['property_type_encoded'].values[0]
    return id


In [ ]:
def busca_room_type(nome):
    id = df_room_type[df_room_type['room_type'] == nome]['room_type_encoded'].values[0]
    return id

In [ ]:
# Fazendo a previsao de precos

neighbourhood = busca_neighbourhood("Copacabana")
property_type = busca_property_type("Condominium")
room_type = busca_room_type("Entire home/apt")

accommodates =5
bathrooms =1 
bedrooms =2
beds=2
mes = 8
estacao = estacao_do_ano(mes)

#print(neighbourhood,property_type,room_type, estacao)

parametros = [neighbourhood, property_type, room_type, accommodates, bedrooms, bathrooms, beds, mes, estacao]

# linha que faz a predição do modelo
predict = modelo.predict([parametros])[0]
print(f"O preço da diária é: R$ {predict:.2f}")


In [ ]:
df4 = df_Airbnb[df_Airbnb["neighbourhood_cleansed"] == "Copacabana"]


df4 = df4[["neighbourhood_cleansed", "property_type", "room_type", "price"]]
df4

In [ ]:
df4["price"].describe()

In [ ]:
# Avalie o desempenho do modelo
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_train)
print(f"Erro médio quadrático (KNN): {mse}")